This notebook compares the scores of the local models and the model chain -- **ABLATION STUDY:** remove the "before" predictors and see how much the performance drops

In [2]:
import numpy as np
import pandas as pd
#import itertools
#import random

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import r2_score, accuracy_score
import os

import sys
sys.path.append(r'C:\Users\lenne\OneDrive\Documenten\Master of Statistics and Data Science\2023-2024\Master thesis\Thesis_Sofia_Lennert')
from chaining import Chain

In [3]:
def missingness_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [4]:
possible_paths = [
    'C:/Users/lenne/OneDrive/Documenten/Master of Statistics and Data Science/2023-2024/Master thesis/Thesis_Sofia_Lennert/new_data',
    'C:/Users/anaso/Desktop/SOFIA MENDES/KU Leuven/Master Thesis/Thesis_Sofia_Lennert/new_data'
]

# Define file names
file = 'merged_data.csv'

# Find full paths to the CSV files
path = next((f'{path}/{file}' for path in possible_paths if os.path.exists(f'{path}/{file}')), None)

data = pd.read_csv(path)

# Bin the number of relapses into 0, 1, 2, 3 and 4+ 
def bin_column(value):
    if value in [0, 1, 2, 3]:
        return str(value)
    else:
        return '4+'
data['NRELAP'] = data['NRELAP'].apply(bin_column)

# Resulting DataFrame will have aggregated data from all four datasets based on the specific_column
pd.set_option('display.max_columns', None)
data

,USUBJID,AGE,SEX,RACE,CONTINENT,CESEV,CECONTRT,TOTRELAP,MHCONTRT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE,SMSTDY,NRELAP,NHPT-before,NHPT-2y,NHPT-after_2y,PASAT_2s-before,PASAT_2s-2y,PASAT_2s-after_2y,PASAT_3s-before,PASAT_3s-2y,PASAT_3s-after_2y,SDMT-before,SDMT-2y,T25FW-before,T25FW-2y,T25FW-after_2y,T-before,T-after,P-before,P-after,N-before,N-after,DS,DS_L,DS_R,SLEC_before,SLEC_after,SES_after,SES_before,VAA,BDI-before,BDI-after,EDSS-before,EDSS-2y,EDSS-after_2y,KFSS1-Sensory-2y,KFSS1-Sensory-after_2y,KFSS1-Sensory-before,KFSS1-Brain-2y,KFSS1-Brain-after_2y,KFSS1-Brain-before,KFSS1-Bowel-2y,KFSS1-Bowel-after_2y,KFSS1-Bowel-before,KFSS1-Pyramidal-2y,KFSS1-Pyramidal-after_2y,KFSS1-Pyramidal-before,KFSS1-Cerebral-2y,KFSS1-Cerebral-after_2y,KFSS1-Cerebral-before,KFSS1-Visual-2y,KFSS1-Visual-after_2y,KFSS1-Visual-before,KFSS1-Cerebellar-2y,KFSS1-Cerebellar-after_2y,KFSS1-Cerebellar-before,KFSS_M-2y,KFSS_M-after_2y,KFSS_M-before,KFSS_P-2y,KFSS_P-after_2y,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,M_R36-SF12-after,P_R36-SF12-after,R36-SF12-after_Ind
0,MSOAC/0649,NaN,F,WHITE,NORTH AMERICA,NaN,NaN,NaN,Y,SPMS,0,1,0,1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.35,10.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MSOAC/2224,38.0,F,WHITE,EUROPE,NaN,NaN,NaN,NaN,RRMS,0,1,0,0,NaN,0.0,27.80,26.55,NaN,NaN,NaN,NaN,55.0,56.0,NaN,38.0,42.0,8.90,8.85,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,30.0,30.0,1.25,1.25,NaN,0.380952,0.388889,3.00,3.0,NaN,0.000000,NaN,0.000000,0.2,NaN,0.2,0.000000,NaN,0.000000,0.5,NaN,0.500000,0.0,NaN,0.0,0.000000,NaN,0.0,0.0,NaN,0.0,0.148148,NaN,0.148148,0.000000,NaN,0.000000,0.733333,0.653846,0.0,0.566667,0.576923,0.0
2,MSOAC/0576,50.0,F,WHITE,NaN,NaN,NaN,NaN,Y,PPMS,0,1,0,1,NaN,0.0,17.75,18.65,17.90,51.0,56.5,56.5,59.0,59.0,59.5,NaN,NaN,4.55,4.70,5.15,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.75,4.0,4.25,0.333333,0.250000,0.333333,0.0,0.0,0.1,0.250000,0.166667,0.333333,0.5,0.500000,0.500000,0.0,0.2,0.0,0.083333,0.083333,0.0,0.4,0.4,0.3,0.240741,0.277778,0.259259,0.166667,0.125000,0.166667,0.771429,0.784810,1.0,0.714286,0.670886,1.0
3,MSOAC/4961,44.0,F,WHITE,NaN,NaN,NaN,NaN,Y,PPMS,0,1,1,1,NaN,0.0,19.75,19.45,20.60,38.5,52.0,54.0,55.5,59.0,58.0,NaN,NaN,4.00,4.90,5.60,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.50,3.5,3.50,0.333333,0.250000,0.333333,0.0,0.1,0.0,0.166667,0.250000,0.250000,0.5,0.500000,0.416667,0.0,0.2,0.0,0.000000,0.000000,0.0,0.2,0.3,0.3,0.185185,0.277778,0.222222,0.083333,0.125000,0.125000,0.671429,0.746835,1.0,0.628571,0.658228,1.0
4,MSOAC/5990,52.0,F,WHITE,NaN,NaN,NaN,NaN,Y,PPMS,1,1,0,1,NaN,0.0,21.45,21.35,22.75,22.0,23.5,22.0,35.0,41.5,44.0,NaN,NaN,11.40,16.80,30.20,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,6.0,6.50,0.166667,0.333333,0.166667,0.2,0.2,0.2,0.166667,0.166667,0.166667,0.5,0.666667,0.500000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.4,0.0,0.222222,0.333333,0.185185,0.083333,0.166667,0.083333,0.714286,0.493671,1.0,0.685714,0.556962,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,MSOAC/2501,46.0,F,WHITE,NaN,NaN,NaN,12.0,NaN,SPMS,0,0,0,0,NaN,0.0,29.40,25.65,NaN,14.0,12.0,NaN,17.5,16.0,NaN,NaN,NaN,9.30,8.10,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.785714,0.481013,1.0,0.607143,0.664557,1.0
2461,MSOAC/8672,43.0,F,NaN,NaN,MODERATE,NaN,1.0,NaN,RRMS,0,0,0,0,25.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [5]:
#variables = ['KFSS_M-2y', 'EDSS-2y', 'T25FW-2y', 'NRELAP']# removed KFSS_P-2y for now -- ('SMSTDY' gave a score of -0.03)
variables = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y', 'P_R36-SF12-after', 'M_R36-SF12-after', 
             'SES_after', 'SLEC_after', 'EDSS-after_2y', 'KFSS_M-after_2y', 'KFSS_P-after_2y', 'NRELAP', 'CESEV']

# Extract targets
targets = data[variables]

# Extract features by dropping the target columns
#features = data.drop(variables, axis=1)

columns_to_keep = ['AGE', 'SEX', 'RACE', 'CONTINENT', 'MHDIAGN', 'CARDIO', 'URINARY', 'MUSCKELET', 'FATIGUE', 
                    'PASAT_2s-before', 'PASAT_3s-before', 'SDMT-before', 'BDI-before', 'T-before','P-before','N-before']
# removed: 'NHPT-before', 'T25FW-before', 'SLEC_before','SES_before','EDSS-before', 'KFSS_M-before', 'KFSS_P-before', 'M_R36-SF12-before', 
# 'P_R36-SF12-before', 'R36-SF12-before_Ind',

features = data[columns_to_keep]
object_columns = features.select_dtypes(include=['object'])
features = pd.get_dummies(features, columns=object_columns.columns, dtype=int)

Run MICE

In [6]:
featuresM=features.copy()

#missing_mask = featuresM.isna()
imputer = IterativeImputer(max_iter=10, random_state=42)
imputed_values = imputer.fit_transform(featuresM)

featuresM = pd.DataFrame(imputed_values, columns=featuresM.columns)

In [7]:
selected_columns = featuresM.iloc[:, :-2]

# Compute the range for each column
ranges = selected_columns.apply(lambda x: x.max() - x.min())

print("Range of values for each column (except last two):")
print(ranges)

Range of values for each column (except last two):
AGE                         54.000000
CARDIO                       1.000000
URINARY                      1.000000
MUSCKELET                    1.000000
FATIGUE                      1.000000
PASAT_2s-before             59.202079
PASAT_3s-before             58.500000
SDMT-before                273.974826
BDI-before                   0.825397
T-before                     1.000000
P-before                     1.000261
N-before                     1.000081
SEX_F                        1.000000
SEX_M                        1.000000
RACE_NON-WHITE               1.000000
RACE_WHITE                   1.000000
CONTINENT_ASIA               1.000000
CONTINENT_EURASIA            1.000000
CONTINENT_EUROPE             1.000000
CONTINENT_NORTH AMERICA      1.000000
CONTINENT_OCEANIA            1.000000
CONTINENT_SOUTH AMERICA      1.000000
MHDIAGN_PPMS                 1.000000
dtype: float64


In [8]:
model_data = pd.concat([featuresM, targets], axis=1)

In [9]:
#columns_to_encode = ['NRELAP', 'CESEV']

le1 = LabelEncoder()
le2 = LabelEncoder()

cesev = le1.fit_transform(np.array(model_data['CESEV']))
nrelap = le2.fit_transform(np.array(model_data['NRELAP']))

model_data['CESEV'] = cesev
model_data["CESEV"] = model_data["CESEV"].replace(3, np.nan)

model_data['NRELAP']=nrelap

# Impute missing values
imputer = IterativeImputer(max_iter=10, random_state=42)
imputed_values = imputer.fit_transform(model_data)

# Convert imputed values back to DataFrame
encoded_data = pd.DataFrame(imputed_values, columns=model_data.columns)
encoded_data[(encoded_data["CESEV"] <= -0.5) | (encoded_data["CESEV"] >= 2.5)]

c:\Users\lenne\anaconda3\envs\Thesis\lib\site-packages\sklearn\impute\_iterative.py:801: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


,AGE,CARDIO,URINARY,MUSCKELET,FATIGUE,PASAT_2s-before,PASAT_3s-before,SDMT-before,BDI-before,T-before,P-before,N-before,SEX_F,SEX_M,RACE_NON-WHITE,RACE_WHITE,CONTINENT_ASIA,CONTINENT_EURASIA,CONTINENT_EUROPE,CONTINENT_NORTH AMERICA,CONTINENT_OCEANIA,CONTINENT_SOUTH AMERICA,MHDIAGN_PPMS,MHDIAGN_RRMS,MHDIAGN_SPMS,KFSS_M-2y,KFSS_P-2y,EDSS-2y,T25FW-2y,NHPT-2y,P_R36-SF12-after,M_R36-SF12-after,SES_after,SLEC_after,EDSS-after_2y,KFSS_M-after_2y,KFSS_P-after_2y,NRELAP,CESEV
115,52.0,0.0,1.0,1.0,1.0,24.000000,41.0,-1.810843,0.300000,0.000000,0.000000,0.033079,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.296296,0.166667,4.00,7.20,22.85,0.683544,0.585714,0.876157,5.453287,3.500000,0.296296,0.208333,0.0,-0.676542
245,56.0,0.0,1.0,0.0,1.0,59.000000,60.0,114.223616,0.245516,0.000000,0.000000,0.026190,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.148148,0.083333,3.25,6.00,21.00,0.518987,0.757143,0.976902,51.083325,6.000000,0.148148,0.083333,0.0,2.835446
369,22.0,0.0,1.0,0.0,1.0,48.053196,60.0,65.500000,0.174603,0.000000,0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.111111,0.000000,2.00,5.05,17.00,0.903846,0.716667,1.250000,15.000000,1.488624,0.117569,0.075643,0.0,-0.619210
405,31.0,0.0,0.0,1.0,0.0,48.268705,60.0,59.817087,0.085421,0.010989,0.003463,0.010325,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.037037,0.041667,1.50,4.05,17.25,0.924051,0.878571,1.250000,25.500000,0.500000,0.018519,0.000000,0.0,-0.566149
411,52.0,0.0,1.0,1.0,0.0,27.000000,27.5,66.786948,0.311856,0.000000,0.000000,0.034415,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.111111,0.083333,3.25,8.40,26.55,0.784810,0.785714,1.126731,15.039336,3.000000,0.185185,0.166667,0.0,-0.592836
515,44.0,0.0,0.0,0.0,1.0,42.500000,55.5,45.956453,0.194849,0.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.108019,0.035845,1.00,5.30,17.65,0.911392,0.871429,1.284064,19.815622,0.000000,0.075700,0.048297,0.0,-0.886980
735,43.0,0.0,0.0,1.0,0.0,37.000000,52.0,25.370683,0.196183,0.000000,0.000000,0.022026,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.296296,0.166667,6.00,9.40,25.15,0.569620,0.535714,1.226407,24.639037,4.000000,0.296296,0.083333,0.0,-0.897229
757,64.0,0.0,1.0,1.0,1.0,34.000000,42.5,41.938328,0.299932,0.000000,0.000000,0.039712,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.222222,0.083333,3.50,6.15,22.90,0.626582,0.592857,1.116034,17.164659,3.250000,0.240741,0.125000,0.0,-0.501415
805,62.0,1.0,1.0,1.0,1.0,46.000000,55.5,55.597030,0.263508,0.000000,0.000000,0.034219,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.314815,0.416667,4.00,6.45,37.50,0.493671,0.571429,0.196345,10.293910,8.000000,0.314815,0.541667,0.0,2.780329
817,40.0,0.0,1.0,1.0,1.0,36.500000,56.0,9.390538,0.252011,0.000000,0.000000,0.019492,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.296296,0.083333,3.00,5.70,20.25,0.607595,0.500000,0.790372,-1.358210,3.000000,0.296296,0.250000,0.0,-0.876150


In [10]:
encoded_data.loc[encoded_data['CESEV'] < -0.5, 'CESEV'] = 0

cesev = np.array(encoded_data['CESEV']).round().astype(int)
nrelap = np.array(encoded_data['NRELAP']).round().astype(int)

print(np.unique(cesev))
print(np.count_nonzero(cesev == -2147483648))
print(np.count_nonzero(cesev == 3))

[0 1 2 3]
0
6


In [11]:
def replace_negative(arr):
    return np.where(arr == -2147483648, 0, arr)

def replace_three(arr):
    return np.where(arr == 3, 2, arr)

cesev = replace_negative(cesev)
cesev = replace_three(cesev)


print(np.unique(cesev))
print(np.count_nonzero(cesev == -2147483648))
print(np.count_nonzero(cesev == 3))

[0 1 2]
0
0


In [12]:
encoded_data['CESEV'] = le1.inverse_transform(cesev)
encoded_data['NRELAP'] = le2.inverse_transform(nrelap)

#print(encoded_data['CESEV'])
#print(encoded_data['NRELAP'])
encoded_data['CESEV'].unique()
targetsM = encoded_data[targets.columns]

In [13]:
selected_columns = targetsM.iloc[:, :-2]

# Compute the range for each column
ranges = selected_columns.apply(lambda x: x.max() - x.min())

print("Range of values for each column (except last two):")
print(ranges)

Range of values for each column (except last two):
KFSS_M-2y             0.759259
KFSS_P-2y             0.750000
EDSS-2y               8.000000
T25FW-2y            177.200000
NHPT-2y             289.100000
P_R36-SF12-after      0.750000
M_R36-SF12-after      1.000000
SES_after             1.409482
SLEC_after           89.077187
EDSS-after_2y         9.043740
KFSS_M-after_2y       0.769191
KFSS_P-after_2y       0.939040
dtype: float64


5-Fold CV

In [14]:
# Set random state for reproducibility
random_state = 42
N_FOLDS = 5

# Generate CV folds
cv=missingness_stratified_cv(features, N_FOLDS, random_state)
cv = cv.to_frame(name="CV Fold")

featuresM_cv = pd.merge(featuresM, pd.DataFrame(cv), left_index=True, right_index=True)
targetsM_cv = pd.merge(targetsM, pd.DataFrame(cv), left_index=True, right_index=True)
targets_cv = pd.merge(targets, pd.DataFrame(cv), left_index=True, right_index=True)

featuresM_cv['CV Fold'].value_counts()

CV Fold
4.0    510
3.0    502
0.0    500
1.0    495
2.0    458
Name: count, dtype: int64

---

# Local Models

In [15]:
y_pred_list = []
y_test_list = []

for i in range(0, N_FOLDS): 
    Xi_train = featuresM_cv[featuresM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = featuresM_cv[featuresM_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targetsM_cv[targetsM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate=False, #RUN LOCAL MODELS
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_list.append(pd.DataFrame(y_pred, columns=yi_test.columns, index=yi_test.index))
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [16]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

1st index: fold, 2nd index: outcome

In [17]:
# Initialize a list to store scores
scores = []
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: #CHANGE VAR LIST HERE
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(y_test_cv)):
        y_test = y_test_cv[fold_index][variables.index(variable_name)] #CHANGE VAR LIST HERE
        y_pred = y_pred_cv[fold_index][variables.index(variable_name)] #CHANGE VAR LIST HERE
        
        # Check if the target variable is numerical or categorical
        if y_test.dtype.kind in 'bifc':
            score = r2_score(y_test, y_pred)
        else:
            score = accuracy_score(y_test, y_pred)
                  
        variable_scores.append(score)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (local):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

Scores for each outcome (local):
KFSS_M-2y: 0.41 (± 0.04)
KFSS_P-2y: 0.31 (± 0.05)
EDSS-2y: 0.57 (± 0.03)
T25FW-2y: -0.09 (± 0.13)
NHPT-2y: -0.00 (± 0.14)
P_R36-SF12-after: 0.36 (± 0.04)
M_R36-SF12-after: 0.23 (± 0.06)
SES_after: 0.09 (± 0.06)
SLEC_after: 0.10 (± 0.04)
EDSS-after_2y: 0.48 (± 0.06)
KFSS_M-after_2y: 0.36 (± 0.09)
KFSS_P-after_2y: 0.06 (± 0.11)
NRELAP: 0.62 (± 0.02)
CESEV: 0.52 (± 0.03)


*Questions/Notes*: 
- using pred="true" gives always score 1 (weird)

---

# Model Chain

### Propagate predicted values

In [18]:
y_pred_list = []
y_test_list = []

for i in range(0, N_FOLDS): 
    Xi_train = featuresM_cv[featuresM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = featuresM_cv[featuresM_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targetsM_cv[targetsM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="pred", #RUN MODELS IN A CHAIN
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_list.append(pd.DataFrame(y_pred, columns=yi_test.columns, index=yi_test.index))
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [19]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

In [20]:
# Initialize a list to store scores
scores = []
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(y_test_cv)):
        y_test = y_test_cv[fold_index][variables.index(variable_name)] 
        y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
        
        # Check if the target variable is numerical or categorical
        if y_test.dtype.kind in 'bifc':
            score = r2_score(y_test, y_pred)
        else:
            score = accuracy_score(y_test, y_pred)
                  
        variable_scores.append(score)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (chain - predicted values):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

Scores for each outcome (chain - predicted values):
KFSS_M-2y: 0.41 (± 0.04)
KFSS_P-2y: 0.30 (± 0.04)
EDSS-2y: 0.54 (± 0.04)
T25FW-2y: -0.01 (± 0.04)
NHPT-2y: 0.07 (± 0.04)
P_R36-SF12-after: 0.35 (± 0.05)
M_R36-SF12-after: 0.19 (± 0.04)
SES_after: 0.12 (± 0.06)
SLEC_after: 0.02 (± 0.08)
EDSS-after_2y: 0.38 (± 0.06)
KFSS_M-after_2y: 0.31 (± 0.08)
KFSS_P-after_2y: 0.04 (± 0.05)
NRELAP: 0.63 (± 0.01)
CESEV: 0.52 (± 0.03)


### Propagate true values

Still need to fix clf in chaining.py for propagate="true" -- do not run the code below, it won't work

In [21]:
y_pred_list = []
y_test_list = []

for i in range(0, N_FOLDS): 
    Xi_train = featuresM_cv[featuresM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = featuresM_cv[featuresM_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targetsM_cv[targetsM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="true", #RUN MODELS IN A CHAIN
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_list.append(pd.DataFrame(y_pred, columns=yi_test.columns, index=yi_test.index))
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [22]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

In [23]:
# Initialize a list to store scores
scores = []
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(y_test_cv)):
        y_test = y_test_cv[fold_index][variables.index(variable_name)] 
        y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
        
        # Check if the target variable is numerical or categorical
        if y_test.dtype.kind in 'bifc':
            score = r2_score(y_test, y_pred)
        else:
            score = accuracy_score(y_test, y_pred)
                  
        variable_scores.append(score)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (chain - true values):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

Scores for each outcome (chain - true values):
KFSS_M-2y: 0.41 (± 0.04)
KFSS_P-2y: 0.31 (± 0.04)
EDSS-2y: 0.55 (± 0.03)
T25FW-2y: 0.02 (± 0.05)
NHPT-2y: 0.08 (± 0.04)
P_R36-SF12-after: 0.37 (± 0.04)
M_R36-SF12-after: 0.24 (± 0.05)
SES_after: 0.12 (± 0.04)
SLEC_after: 0.07 (± 0.10)
EDSS-after_2y: 0.47 (± 0.05)
KFSS_M-after_2y: 0.37 (± 0.07)
KFSS_P-after_2y: 0.08 (± 0.10)
NRELAP: 0.61 (± 0.01)
CESEV: 0.50 (± 0.05)
